# Kết nối Elasticsearch

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from indexMapping import indexMapping


es = Elasticsearch(
"http://localhost:9200",
)
client = es.info()
print(client)

c:\Users\Song\anaconda3\envs\agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'name': '197a127364fe', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'F8nIYeTDTp6xIkyVHNol4w', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


# Lọc CSV và lưu vào cleaned_data.csv

In [ ]:

df = pd.read_csv("../info/info.csv")
df = df.drop_duplicates()
df = df.dropna()
df.to_csv("../info/cleaned_data.csv", index=False)

# Lưu vào trong elasticsearch 1 cột summary lại các nội dung của dữ liệu. Khi đó truy vấn vào vector này nhưng ta có thể trả về toàn bộ thông tin khác 

In [4]:
embed_model = SentenceTransformer('all-mpnet-base-v2')

In [11]:

df["SummaryVector"] = df.apply(lambda row: f"Giá nhà là: {row['Price']} có diện tích: {row['Area']}. \
    ở tại vị trí: {row['Position']} là 1 ngôi nhà dạng: {row['Type']} có những tiện ích như: {row['Benefit']}.", axis=1)

df["SummaryVector"] = df["SummaryVector"].apply(lambda x: embed_model.encode(x))

# Xóa document nếu muốn làm lại từ đầu 

In [10]:
index_name = 'rag'
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")
es.indices.create(index=index_name, mappings=indexMapping)

Index 'rag' deleted.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag'})

# Lưu các index và Elasticsearch document

In [12]:
record_list = df.to_dict("records")
for record in record_list:
    try:
        es.index(index="rag", document=record)
    except Exception as e:
        print(e)


# Lấy các giá trị dựa vào query vector

In [ ]:
max_candi = es.count(index="rag")["count"]
if max_candi > 0:
    query = {
    "field" : "SummaryVector",
    "query_vector" : embed_model.encode("Giá nhà tại Long Biên"),
    "k" : 5,
    "num_candidates" : max_candi , 
}
res = es.knn_search(index="rag", knn=query , source=["Price","Area","Position","Type","Benefit"])
results = []

for hit in res['hits']['hits']:
    print(hit['_source'].get('Price'))

14,2 tỷ
13 triệu/tháng
10 triệu/tháng
10 triệu/tháng
15 triệu/tháng


C:\Users\Song\AppData\Local\Temp\ipykernel_31804\3458814327.py:9: GeneralAvailabilityWarning: This API is in technical preview and may be changed or removed in a future release. Elastic will work to fix any issues, but features in technical preview are not subject to the support SLA of official GA features.
  res = es.knn_search(index="rag", knn=query , source=["Price","Area","Position","Type","Benefit"])
C:\Users\Song\AppData\Local\Temp\ipykernel_31804\3458814327.py:9: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="rag", knn=query , source=["Price","Area","Position","Type","Benefit"])
